# <center>OpenAI在线大模型调用及微调方法

## <center>Ch.10 借助Function calling调用外部工具API方法

&emsp;&emsp;尽管Function calling功能从名称上来看是进行外部函数的调用，但实际上“外部函数”的功能绝不仅仅是进行类似陈明函数这种数值运算，很多时候我们可以通过在函数内添加一些外部工具的API来增加函数功能，例如我们可以在函数中封装一些调用外部实时查询天气工具API，从而实现一键实时查询天气的功能；再比如，我们也可以封装一些外部数据库API，从而实现一键查询数据库数据功能；甚至，我们还可以把谷歌搜索“封装”在一个函数内，通过调用谷歌搜索API来对某些信息进行实时搜索。而这些函数一旦作为外部函数“接入”Chat模型，就相当于是给Chat模型增加了一些额外能力，例如实时查询天气数据的能力、直接对数据库操作的能力、借助谷歌搜索实时获取最新信息的能力等，从另一个角度来说，能够实时接入天气数据的LLM本质上就是一个天气查询的智能对话机器人，而能够直接操作数据库的LLM本质上就是一个定制化的SQL代码解释器，而一个接入谷歌搜索、能够获取实时信息的LLM，其实就是一个简易版的New bing聊天机器人，而这才是Function calling的核武级应用场景——围绕大语言模型实现AI应用的敏捷开发。

&emsp;&emsp;本节我们就将在此前封装好的Function calling功能函数基础上，进一步尝试打通Chat模型调用外部工具API的完整链路。本节我们将以实时查询天气的API工具为例，介绍让Chat模型接入天气查询API的方法，而从下一小节开始，我们将进一步介绍企业级环境下一些定制化AI应用的开发案例。

> 这里需要注意，为何有了New bing、Code interpreter类似应用之后我们还需要进行定制化AI开发，不仅是因为需要探索更多的AI应用的可能性，哪怕是相同功能的AI应用，能否高度结合当前业务环境来进行定制化功能的开发，也是直接决定AI应用价值的重要因素。而要定制化开发AI应用程序，Function calling+Chat模型就是非常高效的解决方案。

In [5]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests

同时需要导入我们自定义的函数库gptLearning：

In [6]:
from gptLearning import *

In [4]:
run_conversation

<function gptLearning.run_conversation(messages, functions_list=None, model='gpt-4-0613')>

- 外部工具API

&emsp;&emsp;首先需要简单说明的是，到底什么是外部工具API。API的全称是应用程序接口，例如目前我们在Python环境中使用的ChatCompletion.create函数，实际上就是OpenAI公司提供的GPT系列模型应用程序的一个接口，通过这个API（create函数或者openai库），我们能够非常顺利的调用GPT模型，并根据输入信息实时输出模型结果。而其他的很多应用程序，也都有相应的API。例如谷歌搜索，也可以通过调用API的形式来实现搜索功能，再比如接下来我们要介绍的OpenWeather API，就是一个可以进行实时天气查询的API。

- OpenWeather与OpenWeather API

&emsp;&emsp;OpenWeather是一家提供全球范围内的气象数据服务的公司，该公司的服务包括实时天气信息、天气预报、历史天气数据以及各种气象相关的报告等，并且OpenWeather开放了一定使用限度内完全免费的API，即我们可以在代码环境中通过调用OpenWeather API来进行实时天气查询、天气预报等功能，这意味着开发者可以将OpenWeather的天气预报功能加入到他们自己的应用或网站中。

> 相比OpenAI，OpenWeather更加Open。

- 极简AI应用开发示例

&emsp;&emsp;当然，一个开放的天气查询API，也意味着我们可以将其封装为一个本地的函数，即通过调用OpenWeather API来获取实时天气信息，然后再用Function calling功能调用该函数，如此一来，就相当于给Chat模型增加了实时获取天气信息的能力，而后我们即可通过Chat模型实时查询天气或根据天气情况询问穿衣建议等，从功能上来说，接入实时天气信息的Chat模型其实也就是一个非常简单的天气查询AI助手。这是一个相对简单的将其他工具API接入Chat模型的示例，接下来我们介绍具体实现方法。

### 1.OpenWeather注册及API key获取方法

&emsp;&emsp;为了能够调用OpenWeather服务，和OpenAI的API使用过程类似，我们首先需要先注册OpenWeather账号，并获取OpenWeather API Key。这里需要注意的是，对于大多数在线服务的API来说，都需要通过API key来进行身份验证，尽管OpenWeather相对更加Open，有非常多的免费使用的次数，但身份验证仍然是必要的防止API被滥用的有效手段。OpenWeather API key获取流程如下：

- Step 1.登录OpenWeather官网并点击Sign—>create account完成注册。该网站无需魔法即可直接登录，可以使用国内邮箱或者QQ邮箱均可进行注册，官网地址为：https://openweathermap.org/

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/e9a6286c483ccf21ab96af1972a918b.png" alt="e9a6286c483ccf21ab96af1972a918b" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/577e19b46f4c124b50a0455ccc132e9.png" alt="577e19b46f4c124b50a0455ccc132e9" style="zoom:33%;" />

- Step 2.获取API-key：注册完成后，即可在API keys页面查看当前账户的API key：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/f44ba05a810500a61516caa28a4fcb4.png" alt="f44ba05a810500a61516caa28a4fcb4" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/e497a7ebb4f8d50ff3b6e362e4b569a.png" alt="e497a7ebb4f8d50ff3b6e362e4b569a" style="zoom:33%;" />

一般来说完成注册后，就会有一个已经激活的API-key。和OpenAI一样，OpenWeather的API key也创建多个。

- Step 3.将其设置为环境变量：和OpenAI API key类似，为了方便后续调用，我们也可以直接将OpenWeather API key设置为环境变量，变量名为OPENWEATHER_API_KEY。具体设置环境变量的方法参考Ch.1中OpenAI APkey设置环境变量流程，此处不再赘述。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/aa0f15de151921727c20918fccc8de0.png" alt="aa0f15de151921727c20918fccc8de0" style="zoom:33%;" />

设置完了环境变量之后，接下来即可按照如下方式创建OpenWeather API key变量：

In [24]:
open_weather_key = os.getenv("OPENWEATHER_API_KEY")

需要注意的是，一般来说首次注册用户，首个API key需要等待2-5小时才会被激活，在此期间使用该API key会返回401错误。

### 2.OpenWeather API使用方法

#### 2.1 API功能说明及计费规则

&emsp;&emsp;在获取了OpenWeather API key之后，接下来我们尝试在Python本地环境调用OpenWeather API。关于OpenWeather API的功能、调用方法和免费额度，我们可以在OpenWeather API介绍页面进行查看：https://openweathermap.org/api

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/bad25ddc293aea44495be4eeb94fe6f.png" alt="bad25ddc293aea44495be4eeb94fe6f" style="zoom:33%;" />

对于OpenWeather API来说，最新版的3.0API能够进行实时天气查询、48小时预报、8天天气预报等，并能够查询历史40年的天气信息。可以说是功能功能非常强大的天气查询API，同时对于全部注册用户，每个用户每天有1000次免费查询的额度，超过1000次则按照0.0015美元/次进行计费。

#### 2.2 实时天气查询API调用方法

&emsp;&emsp;在API介绍页面的下面，OpenWeather还非常贴心的给出了不同功能API的调用方法，这里我们先查看实时天气信息的API Doc，该页面包含了借助API进行实时天气信息查询方法：https://openweathermap.org/current

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/a892d6d4e9d75a131dd5c3e77dfc01c.png" alt="a892d6d4e9d75a131dd5c3e77dfc01c" style="zoom:33%;" />

不同于OpenAI提供了Python库作为API接入方式，OpenWeather API则是通过https协议进行通信，这是一种RESTful风格API，通过SSL或TLS协议对通信进行加密，并使用HTTP协议的原生语义进行操作，通过使用不同的HTTP方法（GET、POST、PUT、DELETE等）对资源进行增、删、改、查等操作。相关技术内容属于前端开发范畴，课程中不做过多原理层面介绍，重点需要了解的是如何使用这种RESTful风格API进行信息获取。

#### 2.3 RESTful风格API使用方法

&emsp;&emsp;这里我们需要补充介绍关于RESTful风格API调用方法，包括本节课程需要使用OpenWeather API、以及下一小节涉及的谷歌搜索API和Google Sheets API都是RESTful API。由于我们主要是在python环境中调用这些API，因此重点介绍Python中这类API调用的基本步骤为：      

- Step 1.构建请求：即输入某个特定的端点，也是后续请求发送的目标端点，对于OpenWeather实时查询天气的API来说，这个端点就是https://api.openweathermap.org/data/2.5/weather ；        

- Step 2.设置查询参数：即创建请求相关参数对象，例如如果是调用OpenWeather API进行天气查询，则需要按照官网说明设置以下参数：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/68356917cf86444d13919caa5d22605.png" alt="68356917cf86444d13919caa5d22605" style="zoom:33%;" />

这些参数分别是城市名称或者位置坐标、API key、返回结果的格式、天气相关单位标准以及输出天气信息的语言，在实际调用OpenWeather API时，可以通过灵活调整这些参数的取值，来获得期望的结果；       

- Step 3.发送请求：接下来我们即可借助python中的request库来发生请求，request库是一个专门为发送HTTP请求提供支持的库，借助request库可以非常便捷的进行get或者post请求，并且能够非常便捷的进行返回结果的解析；       

- Step 4.解析请求：等待API服务返回结果之后，即可围绕该结果进行解析，将其转化为当前环境可读的形式。具体返回的结果的格式可以通过mode参数进行设置，默认情况下是JOSN格式。

#### 2.4 利用OpenWeather API获取实时天气信息

&emsp;&emsp;接下来我们通过一个简单的示例，来介绍如何通过OpenWeather API获取实时天气信息：

In [55]:
import requests

# Step 1.构建请求
url = "https://api.openweathermap.org/data/2.5/weather"

# Step 2.设置查询参数
params = {
    "q": "Beijing",               # 查询北京实时天气
    "appid": open_weather_key,    # 输入API key
    "units": "metric",            # 使用摄氏度而不是华氏度
    "lang":"zh_cn"                # 输出语言为简体中文
}

# Step 3.发送GET请求
response = requests.get(url, params=params)

# Step 4.解析响应
data = response.json()

这里需要注意的是，城市名必须输入英文名，否则无法正确识别。接下来查看返回结果。首先我们先查看response结果：

In [56]:
response

<Response [200]>

In [57]:
type(response)

requests.models.Response

在未解析之前，我们只能查看到基本请求结果状态，这里的200代表成功相应，即本次发送请求获得了对应的响应，且响应内容包含在response中。考虑到默认情况下返回结果是JSON格式，因此后续代码使用了response.json()对其进行解析。解析内容如下：

In [58]:
data

{'coord': {'lon': 116.3972, 'lat': 39.9075},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': '阴，多云',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 30.94,
  'feels_like': 29.42,
  'temp_min': 30.94,
  'temp_max': 30.94,
  'pressure': 1002,
  'humidity': 27,
  'sea_level': 1002,
  'grnd_level': 996},
 'visibility': 10000,
 'wind': {'speed': 6.9, 'deg': 359, 'gust': 11.09},
 'clouds': {'all': 86},
 'dt': 1689412632,
 'sys': {'type': 1,
  'id': 9609,
  'country': 'CN',
  'sunrise': 1689368261,
  'sunset': 1689421349},
 'timezone': 28800,
 'id': 1816670,
 'name': 'Beijing',
 'cod': 200}

> 这里需要注意，若API key未激活，则返回的response将显示401，表示本次请求Unauthorized（未经授权）。此外，若response出现404，则表示URL路径无效，需要根据官网提供的信息重新核对URL路径。

能够看出，返回结果非常丰富，包含了天气条件、温度、湿度、风速、天气描述等信息，这里我们可以简单查看当前天气中的即时温度和天气状况：

In [42]:
# 即时温度最高、最低气温
data['main']['temp_min'], data['main']['temp_max']

(29.94, 29.94)

In [43]:
# 天气状况
data['weather'][0]['description']

'阴，多云'

对比手机查看的天气信息不难发现，OpenWeather提供的天气信息非常准确：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/68acab6de9fb8cbeec4ad34139d04c2.jpg" alt="68acab6de9fb8cbeec4ad34139d04c2" style="zoom:15%;" />

当然，关于data中更多天气信息解读，可以参考官网给出的参数解读Fields in API response部分：https://openweathermap.org/current#current_JSON

### 3.借助Function calling实现Chat模型实时天气查询

- 实时天气信息获取函数编写

&emsp;&emsp;接下来，我们尝试编写一个通过OpenWeather API实时获取天气信息的API，并作为Chat可调用的外部函数之一。很明显，为了确保和大语言模型之间的顺畅通信，此时要求函数的输入和输出都是字符串格式。具体函数编写如下：

In [86]:
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

> 再次提示，我们可以通过json.dumps()函数将JSON转化为字符串，反过来我们可以通过json.loads()函数将字符串转化为JSON对象。

简单测试函数能否正常运行：

In [87]:
get_weather('Hangzhou')

'{"coord": {"lon": 120.1614, "lat": 30.2937}, "weather": [{"id": 501, "main": "Rain", "description": "\\u4e2d\\u96e8", "icon": "10d"}], "base": "stations", "main": {"temp": 30.27, "feels_like": 37.27, "temp_min": 25.95, "temp_max": 31.62, "pressure": 992, "humidity": 80}, "visibility": 9927, "wind": {"speed": 2.88, "deg": 30, "gust": 5.2}, "rain": {"1h": 2.73}, "clouds": {"all": 89}, "dt": 1689499325, "sys": {"type": 2, "id": 2033711, "country": "CN", "sunrise": 1689455265, "sunset": 1689505351}, "timezone": 28800, "id": 1808926, "name": "Hangzhou", "cod": 200}'

能够发现函数能够正常运行。此外，需要注意的是，为了确保函数能够顺利和大愿与你模型通信，我们还必须确保模型能够顺利解读OpenWeather天气信息，具体验证过程分以下三步，其一是需要验证Chat模型本身是否知道OpenWeather：

In [79]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "user", "content": "请问你知道OpenWeather么？"}
  ]
)
response.choices[0].message['content']

'是的，我知道OpenWeather。它是一项提供全球气象相关信息的服务。OpenWeather提供各种类型的天气数据，如当前天气，预报，历史数据等。他们还提供各种API，允许开发者在自己的应用程序或网站中集成这项服务。'

其二是需要进一步验证Chat模型能否针对函数结果进行解读，即能否在基于data信息理解基础上，给出一些天气查询问题准确的回答：

> 当然，这里如果Chat模型本身就了解OpenWeather，那么应该是具备对OpenWeather查询结果解读的能力的。

In [80]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": "天气信息来源于OpenWeather API：https://api.openweathermap.org/data/2.5/weather"},
    {"role": "system", "content": "这是今日杭州市的天气：%s" % data},
    {"role": "user", "content": "请问今日杭州天气如何？请用简体中文回答"}
  ]
)
response.choices[0].message['content']

'今日杭州市的天气为中雨，现在的温度是32.17摄氏度，但是感觉像是39.17摄氏度。最低温度预计将会降到23.95摄氏度，最高温度可能会升到32.17摄氏度。空气压力为990，湿度为81%。风速约为3.11，风向是从35度吹来。整体上多云，云量占84%。'

能够发现模型能够理解OpenWeather返回结果，并根据提问对data进行总结并进行回答。

第三步，也是非常关键的一步，就是我们在进行提问时问题里面的城市名称肯定是中文，但OpenAIWeather API要求输入英文名称，尽管我们在函数参数说明中注明了loc函数需要转化成英文，但对于外部函数的参数整理是Chat模型自发进行的整理，外部无法干预，模型能否按照要求、在对话过程中提取参数并将其整理为指定格式，完全依赖模型自身的推理能力，因此这里我们需要检查模型本身是否知道中文输入的这些城市名称对应的英文名称，验证过程如下：

In [35]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "user", "content": "请问，杭州市的英文名称是？"}
  ]
)
response.choices[0].message['content']

'The English name of 杭州市 is Hangzhou.'

发现模型能够顺利完成城市名称的中英文翻译。

最后需要简单验证的是auto_functions能否顺利的创建get_weather函数的functions参数：

In [78]:
functions = auto_functions(functions_list)
functions

[{'name': 'get_weather',
  'description': '查询即时天气函数，根据输入的城市名称，查询对应城市的实时天气',
  'parameters': {'type': 'object',
   'properties': {'loc': {'description': "城市名称，注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'",
     'type': 'string',
     'required': True}}}}]

能否发现，模型提取的函数关键信息非常到位。

- 实时天气查询功能的Function calling实现

&emsp;&emsp;在一系列验证工作结束后，接下来我们直接将这个函数放入函数库（写入functions_list函数列表），然后直接启动外链函数库之后的多轮对话机器人chat_with_model，查看对话效果。

In [67]:
functions_list = [get_weather]

这里我们首先测试在不带入外部函数时，Chat模型能否准确回答当前天气情况；

In [76]:
chat_with_model(prompt="你好")

模型回答: 你好，有什么可以帮助你的吗？


您还有其他问题吗？(输入退出以结束对话):  请问今天杭州天气如何？


模型回答: 对不起，我没有直接获取实时天气的功能。不过，你可以搜索或者使用天气应用来查看实时天气。


您还有其他问题吗？(输入退出以结束对话):  退出


能够发现，在原始状态下，大模型并不具备实时查询天气信息的能力。接下来，我们在对话中允许模型挂载外部函数库functions_list，并再次询问当前天气情况，得到结果如下：

In [73]:
chat_with_model(functions_list, prompt="你好")

发生错误： Expecting value: line 1 column 1 (char 0)
正在重新运行...
模型回答: 你好，有什么可以帮助你的吗？


您还有其他问题吗？(输入退出以结束对话):  请问今天杭州天气如何？


模型回答: 今天杭州的天气为中雨，温度为32.17度，体感温度为39.17度，最低温度为23.95度，最高温度为32.17度，湿度为81%，风速为3.11。


您还有其他问题吗？(输入退出以结束对话):  你知道什么是陈明函数么？


模型回答: 对不起，我无法找到与"陈明函数"相关的信息，可能你提供的信息有误或者该专业术语具有其他别名。能请你提供更多上下文或者详细描述吗？


您还有其他问题吗？(输入退出以结束对话):  退出


能够发现，此时在外部函数库的加持下，此时Chat模型就具备了实时查询天气的能力，能够对天气问题做出准确回答。通过上述对话流程，我们也进一步测试了此前定义的auto_functions、run_conversation都能顺利运行。而在实际执行Function calling功能时，这些辅助函数也确实能极大程度缩短代码流程、提高开发效率。

&emsp;&emsp;至此，我们就完整构建了一个借助Function calling功能来调用外部工具API的示例，不难发现，借助Function calling，只要能够获取并灵活使用外部工具API，就能快速为大语言模型“赋能”，其背后的技术想象空间非常大，就OpenWeather API一项，我们就可以进一步探索为Chat模型增加未来天气预测、历史天气统计等功能，以满足不同场景的应用需求，而每个功能的加入，都相当于是完成了一个简单的AI应用开发，而这也是Function calling功能号称能够大幅加快AI应用开发的根本原因。

### 4.借助Few-shot提升auto_functions执行稳定性

&emsp;&emsp;在这个实时天气问答的智能应用程序编写时，我们也发现了此前定义的auto_functions还存在一个小问题，那就是有的时候auto_functions会判断get_weather函数参数类型是array类型而不是object类型，这种错误的推理并不会导致auto_functions函数执行时报错，但会导致后续的run_conversation函数运行报错。尽管我们并不知道为何gpt-4模型会推理get_weather函数参数类型是array类型，但确实是需要修正的问题。这里我们可以考虑通过Few-shot的方式来大幅提高auto_functions在执行外部函数参数格式推理时的准确性：即通过为模型编写一些JSON Schema编写示例，来提升推理准确率。具体auto_functions修改方法如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/fee6fea7654cd82b6752a102840b955.png" alt="fee6fea7654cd82b6752a102840b955" style="zoom:33%;" />

具体auto_functions修改方法如下：

In [89]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        
        def chen_ming_algorithm(data):
            """
            陈明算法函数，该函数定义了一种特殊的数据集计算过程
            :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
            :return：陈明函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
            """
            df_new = pd.read_json(data)
            res = np.sum(df_new, axis=1) - 1
            return res.to_json(orient='records')
        
        chen_ming_function_description = inspect.getdoc(chen_ming_algorithm)
        
        chen_ming_function_name = chen_ming_algorithm.__name__
        
        chen_ming_function = {"name": "chen_ming_algorithm",
                              "description": "用于执行陈明算法的函数，定义了一种特殊的数据集计算过程",
                              "parameters": {"type": "object",
                                             "properties": {"data": {"type": "string",
                                                                     "description": "执行陈明算法的数据集"},
                                                           },
                                             "required": ["data"],
                                            },
                             }

        
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__

            user_message1 = '以下是某的函数说明：%s。' % chen_ming_function_description +\
                            '根据这个函数的函数说明，请帮我创建一个function对象，用于描述这个函数的基本情况。这个function对象是一个JSON格式的字典，\
                            这个字典有如下5点要求：\
                            1.字典总共有三个键值对；\
                            2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                            3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                            4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                            5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % chen_ming_function_name
            
            
            assistant_message1 = json.dumps(chen_ming_function)
            
            user_prompt = '现在有另一个函数，函数名为：%s；函数说明为：%s；\
                          请帮我仿造类似的格式为当前函数创建一个function对象。' % (function_name, function_description)

            response = openai.ChatCompletion.create(
                              model="gpt-4-0613",
                              messages=[
                                {"role": "user", "name":"example_user", "content": user_message1},
                                {"role": "assistant", "name":"example_assistant", "content": assistant_message1},
                                {"role": "user", "name":"example_user", "content": user_prompt}]
                            )
            functions.append(json.loads(response.choices[0].message['content']))
        return functions
    
    max_attempts = 3
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

> 可能有的时候在学习提示工程时难以想象的是，Few-shot对模型推理能力的提升居然会应用于提升AI应用开发稳定性中。

接下来测试函数效果。我们尝试连续运行5次，auto_functions在推理时能否准确的将parameters识别为object类型：

In [91]:
auto_functions(functions_list)

[{'name': 'get_weather',
  'description': '查询即时天气函数，使用OpenWeather的API，中国的城市需要使用英文名称',
  'parameters': {'type': 'object',
   'properties': {'loc': {'type': 'string', 'description': '需要查询天气的城市的英文名称'}},
   'required': ['loc']}}]

In [92]:
auto_functions(functions_list)

[{'name': 'get_weather',
  'description': '查询即时天气函数，能够返回制定城市的实时天气信息。注意，中国的城市需要用对应城市的英文名称进行查询',
  'parameters': {'type': 'object',
   'properties': {'loc': {'type': 'string',
     'description': '用于表示查询天气的具体城市名称，中国的城市需用其英文名称'}},
   'required': ['loc']}}]

In [93]:
auto_functions(functions_list)

[{'name': 'get_weather',
  'description': '查询即时天气函数',
  'parameters': {'type': 'object',
   'properties': {'loc': {'type': 'string', 'description': '查询天气的具体城市英文名称'}},
   'required': ['loc']}}]

In [94]:
auto_functions(functions_list)

[{'name': 'get_weather',
  'description': '查询即时天气函数，使用OpenWeather API，在传入适当的城市名称（中国城市需使用英文名）后，即可返回该城市的即时天气数据',
  'parameters': {'type': 'object',
   'properties': {'loc': {'type': 'string',
     'description': '需要查询天气的城市名，中国的城市需要对应的英文名称'}},
   'required': ['loc']}}]

In [95]:
auto_functions(functions_list)

[{'name': 'get_weather',
  'description': '查询即时天气函数，通过输入城市名称(英文)查询对应的天气信息',
  'parameters': {'type': 'object',
   'properties': {'loc': {'type': 'string', 'description': '需要查询天气的城市英文名称'}},
   'required': ['loc']}}]

能够发现，此时auto_functions推理稳定性大幅提升。接下来即可更加顺畅的调用chat_with_model进行对话了：

In [97]:
chat_with_model(functions_list, prompt="你好")

模型回答: 你好！有什么我可以帮助你的吗？


您还有其他问题吗？(输入退出以结束对话):  请问今天天津天气如何？


模型回答: 今天天津的天气是晴朗的，温度为32.97摄氏度，相对应的湿度为38%，风速为2m/s。


您还有其他问题吗？(输入退出以结束对话):  相比杭州天气，天津会更热一些么？


模型回答: 是的，根据最新的天气预报，天津的温度为32.97度，而杭州的温度为30.27度，所以天津会比杭州热一些。


您还有其他问题吗？(输入退出以结束对话):  我在北京，一会儿想要出门，要带伞么？


模型回答: 今天北京的天气是晴空万里，无雨预报，所以你无需带伞。但应考虑到天气较热，建议你戴上防晒帽或者打一把太阳伞防晒。


您还有其他问题吗？(输入退出以结束对话):  退出


至此，我们就顺利完成了auto_functions函数的修改，大幅提升其JSON Schema编写时的推理稳定性性能，后续需要在gptLearning.py中更新auto_functions函数。